# POC - Geração Automática de Atas de Reunião com IA

Este notebook implementa uma prova de conceito para geração automática de atas de reunião usando:
1. **Carregamento de áudio** - Suporte a arquivos .mp3, .wav, .m4a
2. **Diarização** - Separação de speakers com pyannote.audio  
3. **Transcrição** - Conversão de áudio para texto com Whisper
4. **Geração de ata** - Processamento com OpenAI API para criar ata estruturada

---

In [ ]:
# Instalação das dependências necessárias
!pip install git+https://github.com/openai/whisper.git -q
!pip install openai -q
!pip install pyannote.audio -q
!pip install torch torchvision torchaudio -q

In [ ]:
import whisper
import os
from openai import OpenAI
from pyannote.audio import Pipeline
import torch
from datetime import datetime
import json

d:\GitHub Projects\tcc-ata-ia\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuração inicial concluída.


## Configuração dos Modelos

In [ ]:
# Configurar a API da OpenAI
# Substitua pela sua chave de API ou use variável de ambiente
OPENAI_API_KEY = "sua-chave-aqui"  # ou os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

# Carregar modelo Whisper
print("Carregando modelo Whisper...")
whisper_model = whisper.load_model("small")  # Pode usar "medium" ou "large" para melhor qualidade
print("Modelo Whisper carregado com sucesso!")

# Configurar pipeline de diarização
print("Configurando pipeline de diarização...")
# Nota: Para usar pyannote, você precisa aceitar os termos em: https://huggingface.co/pyannote/speaker-diarization
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1")
print("Pipeline de diarização configurado!")

Arquivo carregado: ..\data\raw\audio\entrevista_ufs_fm.mp3


## Funções Principais

In [ ]:
def transcribe_audio(audio_path):
    """
    Transcreve áudio usando Whisper
    """
    try:
        print(f"Transcrevendo áudio: {audio_path}")
        result = whisper_model.transcribe(audio_path, language="pt")
        return result["text"]
    except Exception as e:
        print(f"Erro na transcrição: {e}")
        return ""

def perform_diarization(audio_path):
    """
    Realiza diarização (separação de speakers) do áudio
    """
    try:
        print(f"Realizando diarização: {audio_path}")
        diarization = diarization_pipeline(audio_path)
        
        # Converter resultado para formato mais legível
        speakers_info = []
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            speakers_info.append({
                "speaker": speaker,
                "start": turn.start,
                "end": turn.end,
                "duration": turn.end - turn.start
            })
        
        return speakers_info
    except Exception as e:
        print(f"Erro na diarização: {e}")
        return []

Usando dispositivo: cpu


d:\GitHub Projects\tcc-ata-ia\venv\lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcrição concluída:
 A Universidade Federal de Sejip aprovou recentemente a criação do curso de graduação em inteligência artificial. Quem vai falar sobre o assunto a partir de agora com a gente aqui na UFIS FM é o professor Hendrik Macedo do Departamento de Computação da UFIS. Na satisfação ouvir o professor aqui na UFIS FM como surgiu a proposta de criação desse curso de bacharelado em IA. Boa tarde. Boa tarde, Josaphá. Eu e o que agradeço a oportunidade de esclarecer esse assunto. Veja bem, nós temos associados 


## Geração de Ata com OpenAI

In [ ]:
def generate_meeting_minutes(transcription, speakers_info=None):
    """
    Gera ata de reunião usando OpenAI API
    """
    try:
        # Preparar informações de speakers se disponível
        speaker_context = ""
        if speakers_info:
            unique_speakers = list(set([s["speaker"] for s in speakers_info]))
            speaker_context = f"\n\nParticipantes identificados: {', '.join(unique_speakers)}"
        
        system_prompt = """Você é um assistente especializado em gerar atas de reunião. 
        Sua tarefa é analisar a transcrição fornecida e criar uma ata estruturada e professional.
        
        A ata deve conter:
        1. Cabeçalho com data e participantes
        2. Resumo executivo dos principais pontos
        3. Tópicos discutidos organizados por assunto
        4. Decisões tomadas e responsáveis
        5. Próximos passos e prazos
        6. Observações adicionais se necessário
        
        Mantenha um tom formal e objetivo. Organize as informações de forma clara e hierárquica."""
        
        user_prompt = f"""Transcrição da reunião:
        {transcription}
        {speaker_context}
        
        Por favor, gere uma ata completa e bem estruturada baseada nesta transcrição."""
        
        response = client.chat.completions.create(
            model="gpt-4",  # ou "gpt-3.5-turbo" para economia
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # Baixa criatividade para manter precisão
            max_tokens=2000
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"Erro na geração da ata: {e}")
        return ""

d:\GitHub Projects\tcc-ata-ia\venv\lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [ ]:
def process_meeting_audio(audio_path):
    """
    Função principal que processa o áudio completo:
    1. Carregamento do áudio
    2. Diarização
    3. Transcrição
    4. Geração da ata
    """
    print(f"=== Processando reunião: {audio_path} ===")
    print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Verificar se arquivo existe
    if not os.path.exists(audio_path):
        print(f"Erro: Arquivo não encontrado - {audio_path}")
        return None
    
    # Etapa 1: Diarização
    print("\n1. Realizando diarização...")
    speakers_info = perform_diarization(audio_path)
    print(f"   Encontrados {len(set([s['speaker'] for s in speakers_info]))} speakers diferentes")
    
    # Etapa 2: Transcrição
    print("\n2. Transcrevendo áudio...")
    transcription = transcribe_audio(audio_path)
    print(f"   Transcrição concluída: {len(transcription)} caracteres")
    
    # Etapa 3: Geração da ata
    print("\n3. Gerando ata de reunião...")
    meeting_minutes = generate_meeting_minutes(transcription, speakers_info)
    
    # Resultados
    results = {
        "audio_file": audio_path,
        "processing_date": datetime.now().isoformat(),
        "speakers_info": speakers_info,
        "transcription": transcription,
        "meeting_minutes": meeting_minutes
    }
    
    print(f"\n=== Processamento concluído em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===")
    return results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entidades identificadas:
{'entity_group': 'Organizacao', 'score': 0.8274329, 'word': 'Universidade Federal de Sejip', 'start': 2, 'end': 31}
{'entity_group': 'Organizacao', 'score': 0.5980847, 'word': 'UF', 'start': 182, 'end': 184}
{'entity_group': 'Organizacao', 'score': 0.4919821, 'word': '##IS', 'start': 184, 'end': 186}
{'entity_group': 'Pessoa', 'score': 0.9830049, 'word': 'Hend', 'start': 204, 'end': 208}
{'entity_group': 'Pessoa', 'score': 0.9813471, 'word': '##ri', 'start': 208, 'end': 210}
{'entity_group': 'Pessoa', 'score': 0.96631706, 'word': '##k Macedo', 'start': 210, 'end': 218}
{'entity_group': 'Organizacao', 'score': 0.50499254, 'word': 'UF', 'start': 252, 'end': 254}
{'entity_group': 'Organizacao', 'score': 0.5281395, 'word': '##IS', 'start': 254, 'end': 256}
{'entity_group': 'Organizacao', 'score': 0.6223447, 'word': 'UF', 'start': 298, 'end': 300}
{'entity_group': 'Organizacao', 'score': 0.54624575, 'word': '##IS', 'start': 300, 'end': 302}
{'entity_group': 'Pessoa'

## Teste da POC

In [ ]:
# Teste com um arquivo de áudio das reuniões CONEPE/CONSU
# Ajuste o caminho conforme necessário
audio_file = "../data/raw/audio/conepe/2024-01-22_conepe_#52.wav"

# Verificar se arquivo existe antes de processar
if os.path.exists(audio_file):
    print(f"Processando arquivo: {audio_file}")
    results = process_meeting_audio(audio_file)
    
    if results:
        print("\n" + "="*80)
        print("RESULTADOS DA POC")
        print("="*80)
        
        print(f"\nArquivo processado: {results['audio_file']}")
        print(f"Data de processamento: {results['processing_date']}")
        
        print(f"\nSpeakers identificados: {len(set([s['speaker'] for s in results['speakers_info']]))}")
        for speaker in set([s['speaker'] for s in results['speakers_info']]):
            total_time = sum([s['duration'] for s in results['speakers_info'] if s['speaker'] == speaker])
            print(f"  - {speaker}: {total_time:.1f}s")
        
        print(f"\nTranscrição ({len(results['transcription'])} caracteres):")
        print("-" * 50)
        print(results['transcription'][:500] + "..." if len(results['transcription']) > 500 else results['transcription'])
        
        print(f"\nAta de Reunião:")
        print("-" * 50)
        print(results['meeting_minutes'])
        
        # Salvar resultados
        output_file = f"../data/atas-geradas/ata_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        print(f"\nResultados salvos em: {output_file}")
        
else:
    print(f"Arquivo não encontrado: {audio_file}")
    print("Arquivos disponíveis:")
    audio_dir = "../data/raw/audio"
    if os.path.exists(audio_dir):
        for root, dirs, files in os.walk(audio_dir):
            for file in files:
                if file.endswith(('.wav', '.mp3', '.m4a')):
                    print(f"  - {os.path.join(root, file)}")
    else:
        print("Diretório de áudio não encontrado!")

Resumo gerado:
[{'summary_text': "'Não vão aprender a usar chat APT', diz professor da UFIS. A universidade quer que o curso de inteligência artificial seja um dos mais concorridos do Brasil. E a gente tem uma preocupação com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estrutura física, com a estr

## Notas e Próximos Passos

### Configurações Necessárias

1. **Chave da API OpenAI**: Configure sua chave da API OpenAI na célula de configuração
2. **Hugging Face Token**: Para usar pyannote.audio, você precisa:
   - Criar conta no Hugging Face
   - Aceitar os termos em: https://huggingface.co/pyannote/speaker-diarization
   - Configurar token de acesso

### Melhorias Possíveis

1. **Interface Gradio**: Adicionar interface web para upload de arquivos
2. **Modelos maiores**: Usar Whisper "medium" ou "large" para melhor qualidade
3. **Pós-processamento**: Adicionar correção ortográfica e formatação
4. **Templates**: Criar templates específicos para diferentes tipos de reunião
5. **Exportação**: Gerar PDFs e documentos Word da ata

### Custos Estimados

- **OpenAI API**: ~$0.03-0.06 por minuto de áudio (dependendo do modelo)
- **Processamento local**: Whisper e diarização rodam localmente (gratuito)

In [ ]:
# Função utilitária para listar e testar diferentes arquivos
def list_available_audio_files():
    """Lista todos os arquivos de áudio disponíveis"""
    audio_files = []
    audio_dir = "../data/raw/audio"
    
    if os.path.exists(audio_dir):
        for root, dirs, files in os.walk(audio_dir):
            for file in files:
                if file.endswith(('.wav', '.mp3', '.m4a')):
                    full_path = os.path.join(root, file)
                    rel_path = os.path.relpath(full_path, "..")
                    audio_files.append(rel_path)
    
    return sorted(audio_files)

def quick_test(audio_file_path):
    """Teste rápido com apenas transcrição (sem diarização para economizar tempo)"""
    print(f"Teste rápido: {audio_file_path}")
    
    if not os.path.exists(audio_file_path):
        print(f"Arquivo não encontrado: {audio_file_path}")
        return None
    
    # Apenas transcrição
    transcription = transcribe_audio(audio_file_path)
    
    # Gerar ata sem informação de speakers
    meeting_minutes = generate_meeting_minutes(transcription)
    
    print(f"\nTranscrição ({len(transcription)} chars):")
    print(transcription[:300] + "..." if len(transcription) > 300 else transcription)
    
    print(f"\nAta gerada:")
    print(meeting_minutes)
    
    return {"transcription": transcription, "meeting_minutes": meeting_minutes}

# Listar arquivos disponíveis
print("Arquivos de áudio disponíveis:")
available_files = list_available_audio_files()
for i, file in enumerate(available_files):
    print(f"{i+1:2d}. {file}")

# Exemplo de uso do teste rápido (descomente para usar)
# if available_files:
#     quick_test(available_files[0])